In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("../datasets/titanic/train.csv")

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
mod = data.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)

In [6]:
mod.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
dtype: object

In [7]:
mod["Sex"] = LabelEncoder().fit_transform(mod["Sex"])

In [8]:
mod.dtypes

Survived      int64
Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
dtype: object

In [9]:
mod.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,22.0,1,0
1,1,1,0,38.0,1,0
2,1,3,0,26.0,0,0
3,1,1,0,35.0,1,0
4,0,3,1,35.0,0,0


In [10]:
mod.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000
mean,0.383838,2.308642,0.647587,29.699118,0.523008,0.381594
std,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000
50%,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000


In [11]:
mod_full = mod.dropna()

In [12]:
mod_blank = mod.loc[mod["Age"].isna()]

In [13]:
table = mod_full.groupby(["Sex", "Pclass"]).mean()[["Age"]].reset_index()

In [14]:
row = mod_blank.iloc[0]

table.loc[(table["Sex"] == row["Sex"]) &(table["Pclass"] == row["Pclass"])].iloc[0]["Age"]

26.507588932806325

In [15]:
def fillAge(row):
    row["Age"] = table.loc[(table["Sex"] == row["Sex"]) &(table["Pclass"] == row["Pclass"])].iloc[0]["Age"]
    return row

mod_filled = mod_blank.apply(fillAge, axis=1)

In [16]:
total = pd.concat([mod_full, mod_filled])

In [17]:
X = total.drop(["Survived"], axis=1)
y = total["Survived"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [19]:
from sklearn.tree import DecisionTreeClassifier

In [20]:
model = DecisionTreeClassifier()

In [21]:
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [22]:
model.score(X_test, y_test)

0.7932203389830509

In [24]:
X.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch'], dtype='object')

In [27]:
X["Age"].mean()

29.318642716644238

In [23]:
def entropy(column):
    items, counts = np.unique(column, return_counts=True)
    p = counts / len(column)
    return np.sum(-p*np.log(p))

In [29]:
def info_gain(X, y, label):
    
    pivot = X[label].mean()
    
    left = X[label] < pivot
    right = X[label] >= pivot
    
    y_left, y_right = y.loc[left], y.loc[right]
    
    p_left, p_right = len(y_left)/len(y), len(y_right)/len(y)
    
    return entropy(y) - p_left*entropy(y_left) - p_right*entropy(y_right)
    

In [30]:
for col in X.columns:
    print(col, info_gain(X, y, col))

Pclass 0.05253613916173083
Sex 0.15087048925218172
Age 0.0007663481843047659
SibSp 0.006643498134917131
Parch 0.010661126611803101


In [37]:
class Node:
    
    def __init__(self, label=None, pivot=None, result=None):
        self.label = label
        self.pivot = pivot
        self.result = result
        
        self.left = None
        self.right = None
        
    def __repr__(self):
        
        if self.label == None:
            return str(self.result > .5)
        else:
            return str(self.label + " : " + str(self.pivot))
        

In [47]:
class DecisionTreeImpl:
    
    
    def __init__(self, max_depth=5):
        self.max_depth = max_depth

        
    def fit(self, X, y):
        self.root = self.fit_rec(X, y, max_depth=self.max_depth)
        
    def fit_rec(self, X, y, max_depth):
        if max_depth == 0:
            return Node(result=y.mean())
        
        gains = []
        for col in X.columns:
            gains.append((info_gain(X, y, col), col))
        
        selected_gain, selected_col = sorted(gains)[-1]
        
        if selected_gain <= 0:
            return Node(result=y.mean())
        
        pivot = X[selected_col].mean()
        left = X[selected_col] < pivot
        right = X[selected_col] >= pivot
        X_left, X_right = X.loc[left], X.loc[right]
        y_left, y_right = y.loc[left], y.loc[right]
        
        node = Node(label=selected_col, pivot=pivot)
        node.left = self.fit_rec(X_left, y_left, max_depth-1)
        node.right = self.fit_rec(X_right, y_right, max_depth-1)
        return node
        
    
    def display(self):
        self.display_rec(self.root)
        
    def display_rec(self, node, indent=""):
        if node == None:
            return
        
        print(indent, node)
        self.display_rec(node.left, indent + "\t")
        self.display_rec(node.right, indent + "\t")
    

In [48]:
model = DecisionTreeImpl()

In [49]:
model.fit(X_train, y_train)

In [50]:
model.display()

 Sex : 0.6593959731543624
	 Pclass : 2.147783251231527
		 Pclass : 1.4272727272727272
			 Parch : 0.5079365079365079
				 Age : 35.84017216642754
					 True
					 True
				 Parch : 1.4545454545454546
					 True
					 True
			 Parch : 0.5531914893617021
				 Age : 32.68153153153153
					 True
					 True
				 Age : 25.764705882352942
					 True
					 True
		 Parch : 0.7634408602150538
			 Age : 23.514150943396228
				 SibSp : 0.225
					 True
					 True
				 SibSp : 0.3076923076923077
					 False
					 False
			 SibSp : 1.925
				 Age : 23.09375
					 True
					 False
				 Age : 12.8125
					 False
					 False
	 Pclass : 2.3587786259541983
		 Pclass : 1.4819277108433735
			 Age : 41.73238314529128
				 SibSp : 0.23529411764705882
					 False
					 True
				 Age : 53.6
					 False
					 False
			 Parch : 0.2125
				 Age : 32.851287113559835
					 False
					 False
				 Age : 12.344999999999999
					 True
					 False
		 Parch : 0.23788546255506607
			 Age : 28.71426691223642
				 Age : 23.